# Application: Car license plate Detection and estimate speed with Yolo8 and Opencv

In [1]:
import string
import easyocr
reader = easyocr.Reader(['en'], gpu=False)
# Mapping dictionaries for character conversion
dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}

dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}
def license_complies_format(text):
    """
    Check if the license plate text complies with the required format.

    Args:
        text (str): License plate text.

    Returns:
        bool: True if the license plate complies with the format, False otherwise.
    """
    if len(text) != 7:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False


def format_license(text):
    """
    Format the license plate text by converting characters using the mapping dictionaries.

    Args:
        text (str): License plate text.

    Returns:
        str: Formatted license plate text.
    """
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_


def read_license_plate(license_plate_crop):
    """
    Read the license plate text from the given cropped image.

    Args:
        license_plate_crop (PIL.Image.Image): Cropped image containing the license plate.

    Returns:
        tuple: Tuple containing the formatted license plate text and its confidence score.
    """

    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')

        if license_complies_format(text):
            return format_license(text), score

    return None, None


Using CPU. Note: This module is much faster with a GPU.


In [5]:
from collections import defaultdict, deque
import cv2
import logging
import numpy as np
from ultralytics import YOLO
logging.getLogger('ultralytics').setLevel(logging.WARNING)
import supervision as sv

SOURCE = np.array([[1252, 787], [2298, 803], [5039, 2159], [-550, 2159]])

TARGET_WIDTH = 25
TARGET_HEIGHT = 250

TARGET = np.array(
    [
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ]
)

class ViewTransformer:
    def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points

        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)



if __name__ == "__main__":
    #source_video_path = 'vehicles.mp4'
    source_video_path = 'sample.mp4'
    confidence_threshold = 0.5
    target_video_path = 'output.mp4'
    iou_threshold = 0.7
    license_plate_crop_thresh = 0.4
    video_info = sv.VideoInfo.from_video_path(video_path=source_video_path)
    model = YOLO("yolov8n.pt")
    license_plate_detector = YOLO('license_plate_detector.pt')

    byte_track = sv.ByteTrack(
        frame_rate=video_info.fps, track_thresh=confidence_threshold
    )

    thickness = sv.calculate_optimal_line_thickness(
        resolution_wh=video_info.resolution_wh
    )
    text_scale = sv.calculate_optimal_text_scale(resolution_wh=video_info.resolution_wh)
    bounding_box_annotator = sv.BoundingBoxAnnotator(thickness=thickness)
    label_annotator = sv.LabelAnnotator(
        text_scale=text_scale,
        text_thickness=thickness,
        text_position=sv.Position.BOTTOM_CENTER,
    )
    trace_annotator = sv.TraceAnnotator(
        thickness=thickness,
        trace_length=video_info.fps * 2,
        position=sv.Position.BOTTOM_CENTER,
    )

    frame_generator = sv.get_video_frames_generator(source_path=source_video_path)

    polygon_zone = sv.PolygonZone(polygon=SOURCE)
    view_transformer = ViewTransformer(source=SOURCE, target=TARGET)

    coordinates = defaultdict(lambda: deque(maxlen=video_info.fps))
    tracker_license_plates = {}
    tracker_id_speed = {}

    with sv.VideoSink(target_video_path, video_info) as sink:
        for frame in frame_generator:
            result = model(frame)[0]
            detections = sv.Detections.from_ultralytics(result)
            detections = detections[detections.confidence > confidence_threshold]
            detections = detections[polygon_zone.trigger(detections)]
            detections = detections.with_nms(threshold=iou_threshold)
            detections = byte_track.update_with_detections(detections=detections)
            license_plates = license_plate_detector(frame)[0]
            points_center = detections.get_anchors_coordinates(
                anchor=sv.Position.CENTER
            )
            points = detections.get_anchors_coordinates(
                anchor=sv.Position.BOTTOM_CENTER
            )
            points = view_transformer.transform_points(points=points).astype(int)

            for tracker_id, [_, y] in zip(detections.tracker_id, points):
                coordinates[tracker_id].append(y)

            labels = []
            for tracker_id in detections.tracker_id:
                if len(coordinates[tracker_id]) < video_info.fps / 2:
                    labels.append(f"#{tracker_id}")
                else:
                    coordinate_start = coordinates[tracker_id][-1]
                    coordinate_end = coordinates[tracker_id][0]
                    distance = abs(coordinate_start - coordinate_end)
                    time = len(coordinates[tracker_id]) / video_info.fps
                    speed = distance / time * 3.6
                    labels.append(f"#{tracker_id} {int(speed)} km/h")
                    tracker_id_speed[tracker_id] = int(speed)

            for license_plate in license_plates.boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = license_plate
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

                license_plate_crop = frame[y1:y2, x1:x2, :]

                # Process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # Read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)
                #print(f"Plate: {license_plate_text}, Score: {license_plate_text_score}")
                if license_plate_text is not None and license_plate_text != "":
                    # Associate the license plate text with the closest tracker_id
                    for i, tracker_id in enumerate(detections.tracker_id):
                        x, y = points_center[i]
                        #print(x,y,tracker_id,x1,y1,x2,y2)
                        if x1 <= x <= x2:# and y1 <= y <= y2:
                            tracker_license_plates[tracker_id] = license_plate_text
                            cv2.putText(frame, license_plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
                            break

            if tracker_id in tracker_id_speed and tracker_id in tracker_license_plates:
                print(f"#{tracker_id}: {tracker_license_plates[tracker_id]} speed: {tracker_id_speed[tracker_id]} km/h")
            annotated_frame = frame.copy()
            annotated_frame = trace_annotator.annotate(
                scene=annotated_frame, detections=detections
            )
            annotated_frame = bounding_box_annotator.annotate(
                scene=annotated_frame, detections=detections
            )
            annotated_frame = label_annotator.annotate(
                scene=annotated_frame, detections=detections, labels=labels
            )

            '''for tracker_id, license_plate_text in tracker_license_plates.items():
                for detection in detections:
                    if detection.tracker_id == tracker_id:
                        x1, y1, x2, y2 = detection.xyxy
                        cv2.putText(annotated_frame, license_plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
                        break
            '''
            sink.write_frame(annotated_frame)
            display_frame = cv2.resize(annotated_frame, (1200, 700))
            # Show frame
            cv2.imshow("frame", display_frame)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        cv2.destroyAllWindows()


SupervisionWarnings: `track_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'track_activation_threshold' instead.


#24: NA13NRU speed: 67 km/h
#24: NA13NRU speed: 63 km/h
#26: MU51TSV speed: 84 km/h
#26: CI51VSU speed: 79 km/h
#24: NA13NRU speed: 64 km/h
#24: NA13NRU speed: 66 km/h
#24: NA13MRU speed: 61 km/h
#24: NA13NRU speed: 56 km/h
#24: NA13MRU speed: 54 km/h
#24: MA13NRU speed: 52 km/h
#24: NA13NRU speed: 50 km/h
#24: NA13MRU speed: 48 km/h
#24: NA13NRU speed: 46 km/h
#24: NA13NRU speed: 43 km/h
#26: HU51YSU speed: 75 km/h
#24: NA13NRU speed: 28 km/h
#26: HU51MSU speed: 72 km/h
#30: GX15OGJ speed: 100 km/h
#30: GX15OCJ speed: 86 km/h
#26: MV51VSV speed: 39 km/h
#30: GX15OGJ speed: 90 km/h
#26: HL51VSU speed: 28 km/h
#26: HU51NSU speed: 25 km/h
#26: HY51VSI speed: 21 km/h
#30: GX15OGJ speed: 75 km/h
#30: GX15OCJ speed: 79 km/h
#30: GX15OGJ speed: 75 km/h
#30: CX15OGJ speed: 82 km/h
